### Description
This is a movie prediction competition, we need to recommend movies for users based on their reviews and information about them

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's severabl helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rec-sys-challenge-task-course-2023/train.csv
/kaggle/input/rec-sys-challenge-task-course-2023/kaggle_baseline.csv


### Load and prepare data

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

# Load data
df = pd.read_csv('/kaggle/input/rec-sys-challenge-task-course-2023/train.csv')
df.head()

2024-06-28 12:34:13.567694: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-28 12:34:13.567847: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-28 12:34:13.729480: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


,user_id,title,movie_id,rating,release_date,sex,age
0,2592,Top Gun (1986),1101,4,Action|Romance,50,M
1,4318,12 Angry Men (1957),1203,4,Drama,25,M
2,2756,Robocop 2 (1990),2986,2,Action|Crime|Sci-Fi,18,M
3,1706,Modern Times (1936),3462,5,Comedy,25,M
4,4813,Milk Money (1994),276,3,Comedy|Romance,35,F


In [3]:
df.rename(columns={'age': 'gender'}, inplace=True)
df.rename(columns={'sex': 'age'}, inplace=True)
df.rename(columns={'gender': 'sex'}, inplace=True)
df.head()

,user_id,title,movie_id,rating,release_date,age,sex
0,2592,Top Gun (1986),1101,4,Action|Romance,50,M
1,4318,12 Angry Men (1957),1203,4,Drama,25,M
2,2756,Robocop 2 (1990),2986,2,Action|Crime|Sci-Fi,18,M
3,1706,Modern Times (1936),3462,5,Comedy,25,M
4,4813,Milk Money (1994),276,3,Comedy|Romance,35,F


In [4]:
user_ids = df['user_id'].unique().tolist()
movie_ids = df['movie_id'].unique().tolist()

user_id_to_index = {id: i for i, id in enumerate(user_ids)}
movie_id_to_index = {id: i for i, id in enumerate(movie_ids)}

df['user_index'] = df['user_id'].map(user_id_to_index)
df['movie_index'] = df['movie_id'].map(movie_id_to_index)

X = df[['user_index', 'movie_index']].values
y = df['rating'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Define and train model

In [5]:
user_input = Input(shape=(1,), name='user_input')
movie_input = Input(shape=(1,), name='movie_input')

user_embedding = Embedding(input_dim=len(user_ids), output_dim=50, name='user_embedding')(user_input)
movie_embedding = Embedding(input_dim=len(movie_ids), output_dim=50, name='movie_embedding')(movie_input)

user_vec = Flatten(name='flatten_users')(user_embedding)
movie_vec = Flatten(name='flatten_movies')(movie_embedding)

concat = Concatenate(name='concat')([user_vec, movie_vec])
dense = Dense(128, activation='relu', name='dense')(concat)
output = Dense(1, name='output')(dense)

model = Model(inputs=[user_input, movie_input], outputs=output)

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit([X_train[:, 0], X_train[:, 1]], y_train, epochs=10, batch_size=64, validation_data=([X_test[:, 0], X_test[:, 1]], y_test))

Epoch 1/10
10003/10003 ━━━━━━━━━━━━━━━━━━━━ 43s 4ms/step - loss: 1.3811 - val_loss: 0.8256
Epoch 2/10
10003/10003 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step - loss: 0.8083 - val_loss: 0.8003
Epoch 3/10
10003/10003 ━━━━━━━━━━━━━━━━━━━━ 42s 4ms/step - loss: 0.7704 - val_loss: 0.7851
Epoch 4/10
10003/10003 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step - loss: 0.7460 - val_loss: 0.7887
Epoch 5/10
10003/10003 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step - loss: 0.7268 - val_loss: 0.7789
Epoch 6/10
10003/10003 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step - loss: 0.7002 - val_loss: 0.7752
Epoch 7/10
10003/10003 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step - loss: 0.6716 - val_loss: 0.7789
Epoch 8/10
10003/10003 ━━━━━━━━━━━━━━━━━━━━ 42s 4ms/step - loss: 0.6414 - val_loss: 0.7812
Epoch 9/10
10003/10003 ━━━━━━━━━━━━━━━━━━━━ 43s 4ms/step - loss: 0.6140 - val_loss: 0.7929
Epoch 10/10
10003/10003 ━━━━━━━━━━━━━━━━━━━━ 42s 4ms/step - loss: 0.5882 - val_loss: 0.8079


### Generate predictions for the test dataset and create a submission file

In [6]:
test = pd.read_csv('/kaggle/input/rec-sys-challenge-task-course-2023/kaggle_baseline.csv')
test

,user_id,prediction
0,850,260 1210 480 2028 589 1270 593 1580 110 2396 1...
1,2012,2858 2028 593 608 2762 110 2396 1617 527 2997 ...
2,1511,2858 260 480 589 1270 593 1580 608 2762 110 23...
3,5846,1196 1210 480 2028 589 2571 1270 1580 1198 276...
4,4344,593 2716 1 3578 2987 919 3175 924 1387 2355 10...
...,...,...
6032,421,2858 1196 260 480 2028 589 2571 1270 593 1580 ...
6033,4393,1196 260 480 2028 589 1270 593 1580 608 1198 2...
6034,3455,1196 260 1210 480 2028 589 2571 1270 593 1580 ...
6035,2714,1196 1210 480 2028 589 2571 1270 593 1198 2762...


In [7]:
test['prediction'] = test['prediction'].apply(lambda x: x.split())

test_user_indices = []
test_movie_indices = []

for index, row in test.iterrows():
    user_index = user_id_to_index[row['user_id']]
    for movie_id in row['prediction']:
        movie_index = movie_id_to_index[int(movie_id)]
        test_user_indices.append(user_index)
        test_movie_indices.append(movie_index)

test_user_indices = np.array(test_user_indices)
test_movie_indices = np.array(test_movie_indices)

predictions = model.predict([test_user_indices, test_movie_indices])

submission = pd.DataFrame({
    'user_id': test['user_id'].repeat(test['prediction'].apply(len)),
    'movie_id': [int(movie_id) for sublist in test['prediction'] for movie_id in sublist],
    'prediction': predictions.flatten()
})

submission = submission.groupby('user_id').apply(lambda x: x.nlargest(25, 'prediction')).reset_index(drop=True)
submission = submission[['user_id', 'movie_id']].groupby('user_id').agg(lambda x: ' '.join(map(str, x))).reset_index()
submission.columns = ['user_id', 'prediction']

submission.to_csv('submission.csv', index=False)

4717/4717 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step


/tmp/ipykernel_18/1654746605.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  submission = submission.groupby('user_id').apply(lambda x: x.nlargest(25, 'prediction')).reset_index(drop=True)
